In [121]:
#Task 2

import numpy as np
import pandas as pd
import scipy.stats as stats
from multiprocessing import Pool


# Simulation parameters
n_simulations = 10**4  # Number of simulations to approximate the distribution
beta = 10**8  # Inverse temperature
lambda_step = 10**(-4)  # Step size for SGLD
gamma = 10**(-8)  # Regularization parameter
theta = 0  # Initial guess for theta

# Set quantile levels and number of iterations for SGLD
q = 0.95
n_iterations = 10**5
n_assets = 3  # Number of assets in the portfolio
# Define the initial weights for the portfolio
initial_weights = np.array([1.0 / n_assets] * n_assets)

# Asset parameters for each scenario

asset_params = [
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 1}],
    [{'mu': 0, 'sigma': 10**3}, {'mu': 0, 'sigma': 1}, {'mu': 0, 'sigma': 4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 2, 'sigma': 1}]
]


In [122]:

def softmax(weights):
    """Compute softmax values for each set of scores in x."""
    e_w = np.exp(weights - np.max(weights))
    return e_w / e_w.sum()

def softmax_derivative(w, j):
    """Calculate the softmax derivative for weight index j with respect to all weights."""
    exp_weights = np.exp(w)
    sum_exp_weights = np.sum(exp_weights)
    
    # Initialize an array for the derivatives
    derivative = np.zeros_like(w)
    
    # Compute the derivative for each weight i with respect to weight j
    for i in range(len(w)):
        if i == j:
            derivative[i] = exp_weights[j] * (sum_exp_weights - exp_weights[j]) / sum_exp_weights**2
        else:
            derivative[i] = -exp_weights[i] * exp_weights[j] / sum_exp_weights**2
    
    return derivative

def simulate_asset_returns(mu, sigma, size):
    # Simulate asset returns with truncation to limit extreme values
    return np.clip(np.random.normal(mu, sigma, size), mu - 3*sigma, mu + 3*sigma)

def calculate_portfolio_return(weights, returns):
    """Calculate the weighted sum of returns."""
    return np.dot(returns, weights)

# Placeholder for the loss function V(theta)
def V(theta, returns, q, gamma):
    loss = np.mean([theta + (1/(1 - q)) * max(return_ - theta, 0) for return_ in returns]) + gamma * theta**2
    return loss

def calculate_loss(theta, weights, returns, q, gamma):
    """Calculate the loss function for CVaR optimization."""
    loss = np.mean([theta + (1/(1 - q)) * max(np.dot(return_ , weights)- theta, 0) for return_ in returns]) + gamma * theta**2
    return loss

def H_theta(theta, weights, returns, q, gamma):
    """Compute the gradient with respect to theta."""
    portfolio_return = calculate_portfolio_return(weights, returns)
    indicator = (portfolio_return >= theta).astype(float)
    return 1 - np.mean(indicator) / (1 - q) + 2 * gamma * theta

def H_weights(theta, weights, returns, q, gamma):
    """Compute the gradient with respect to the portfolio weights."""
    n_assets = len(weights)
    soft_weights = softmax(weights)
    
    # Compute portfolio returns
    portfolio_returns = calculate_portfolio_return(soft_weights, returns)
    
    # Compute indicator for whether each simulation's return is below theta
    indicator = (portfolio_returns >= theta).astype(float)

    # Initialize the gradient vector for weights
    grad_w = np.zeros(n_assets)

    # Compute gradients for all weights
    for j in range(n_assets):
        # Compute derivative of the softmax for weight j
        derivative = softmax_derivative(weights, j)
        
        # Sum the contributions to the gradient from all simulations
        for i in range(returns.shape[0]):  # Iterate over simulations
            grad_contribution = derivative * (returns[i] - theta) * indicator[i]
            grad_w[j] += np.sum(grad_contribution) / (1 - q)
        
        # Add regularization term
        grad_w[j] += 2 * gamma * weights[j]

    return grad_w


In [123]:
# 下面是更新 SGLD 步骤的函数
def sgld_step(theta, weights, returns, q, gamma, lambda_step, beta):
    # Assuming returns is of shape (n_simulations, n_assets) and weights is of shape (n_assets,)
    theta_grad = H_theta(theta, weights, returns, q, gamma)
    weights_grad = H_weights(theta, weights, returns, q, gamma)

    # SGLD update for theta
    theta += -lambda_step * theta_grad + np.sqrt(2 * lambda_step / beta) * np.random.randn()
    #theta = max(theta, 0)  # Ensure theta is non-negative

    # SGLD update for weights
    weights += -lambda_step * weights_grad + np.sqrt(2 * lambda_step / beta) * np.random.randn(len(weights))
    weights = softmax(weights)  # Normalizing the weights after update
    return theta, weights

def run_sgld_simulation(asset_params, q, lambda_step, gamma, n_iterations, n_simulations, n_assets, beta):
    theta = 0  # Initial random value for theta
    weights = np.array([1.0 / n_assets] * n_assets)  # Initial random weights for assets
    returns = np.array([simulate_asset_returns(param['mu'], param['sigma'], n_simulations) for param in asset_params]).T
    for _ in range(n_iterations):
        # Generate returns for each asset ensuring the shape is (n_simulations, n_assets)
        
        # Update theta and weights using SGLD step
        theta, weights = sgld_step(theta, weights, returns, q, gamma, lambda_step, beta)
    # returns = np.array([simulate_asset_returns(param['mu'], param['sigma'], n_simulations) for param in asset_params]).T
    # Calculate the final CVaR using the optimized parameters
    cvar = calculate_loss(theta, weights, returns, q, gamma)
    return weights, theta, cvar




In [124]:

results = []
for scenario_params in asset_params:
    optimized_weights, VaR, CVaR = run_sgld_simulation(scenario_params, q, lambda_step, gamma, n_iterations, n_simulations, n_assets, beta)
    results.append({
        'Optimized Weights': optimized_weights,
        'VaR_SGLD': VaR,
        'CVaR_SGLD': CVaR
    })

# You can then convert these results to a DataFrame for display similar to the table in your images
results_df = pd.DataFrame(results)
print(results_df)


                                   Optimized Weights  VaR_SGLD  CVaR_SGLD
0                                    [0.0, 0.0, 1.0]  0.000195   0.000214
1                                    [0.0, 0.0, 1.0]  1.667045   2.070490
2                 [0.0, 1.0, 1.412304749076238e-277]  1.624726   2.033299
3  [0.007926618503738074, 1.341881121654952e-06, ...  0.013067   0.016502
4  [0.9688457227275914, 0.0025036704027919477, 0....  1.693397   2.082974
